In [1]:
!pip install torch-geometric -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.1/63.1 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 20.0 MB/s eta 0:00:00


## Imports

In [2]:
import numpy as np
import torch
from torch_geometric.nn import SAGEConv
from torch_geometric.data import Data
import torch.nn.functional as F
import networkx as nx
from torch_geometric.utils import from_networkx
from torch_geometric.utils.convert import from_networkx
import matplotlib.pyplot as plt
from tqdm import tqdm
from scipy.io import mmread

## Graph Robustness Metrics

**1. Effective Graph Resistance (EGR)**  
   $$
   R_g = \frac{2}{N-1} \sum_{i=1}^{N-c} \frac{1}{\lambda_i}
   $$
   where $ \lambda_i $ are the eigenvalues of the Laplacian matrix of the graph.

In [3]:
def compute_effective_resistance(graph):
    laplacian = nx.laplacian_matrix(graph).toarray()
    eigenvalues = np.linalg.eigvalsh(laplacian)
    eigenvalues = eigenvalues[eigenvalues > 1e-8]  # Avoid zero eigenvalues
    N = graph.number_of_nodes()
    return (2 / (N - 1)) * np.sum(1 / eigenvalues)

**2. Weighted Spectrum (WS)**  
   $$
   W_s = \sum_i (1 - \lambda_i)^n
   $$
   where $ n $ controls the depth of analysis

In [4]:
def compute_weighted_spectrum(graph, n=3):
    laplacian = nx.normalized_laplacian_matrix(graph).toarray()
    eigenvalues = np.linalg.eigvalsh(laplacian)
    return np.sum((1 - eigenvalues) ** n)

## Algorithm 1: ILGR Embedding Module
**Input:** Graph $ G $, input node features $ X_v $ $ \forall v \in V $, unknown model weights $ W $ (combination weights) and $ Q $ (aggregation weights).

**Output:** Nodes embedding vector $ z_v $ $ \forall v \in V $.

**1. Initialize**: $ h^0_v = X_v $ for all $ v \in V $.
**2. For each layer** $ l = 1 $ to $ L $ do:
   - For each node $ v = 1 $ to $ V $:
     1. Compute neighborhood embedding using attention mechanism:
        $$
        h^l_{N(v)} = \text{Attention}(Q^l h^{l-1}_k) \quad \forall k \in N(v)
        $$
     2. Compute new embedding for node $ v $ using a **skip connection**:
        $$
        h^l_v = \text{ReLU} \left( W^l \left[ h^{l-1}_v || h^{l-2}_v || h^l_{N(v)} \right] \right)
        $$
**3. Return**: Final embedding vector $ z_v = h^L_v $ for all $ v \in V $.


In [5]:
class ILGRNodeEmbedding(torch.nn.Module):
    def __init__(self, hidden_channels):
        super().__init__()
        self.conv1 = SAGEConv(1, hidden_channels)  # input feature: criticality 
        self.conv2 = SAGEConv(hidden_channels, hidden_channels)
        self.conv3 = SAGEConv(hidden_channels, hidden_channels)
        self.attention = torch.nn.MultiheadAttention(hidden_channels, 1)
        
    def forward(self, x, edge_index):
        # Skip connections and attention
        h1 = torch.relu(self.conv1(x, edge_index))
        h2 = torch.relu(self.conv2(h1, edge_index))
        h3 = torch.relu(self.conv3(h2, edge_index))
        h, _ = self.attention(h3, h3, h3)
        return torch.cat([h1, h2, h3, h], dim=-1)

## Regression Module

The regression module applies a **non-linear transformation** using multiple layers:

$$
y_m = f(W_m \cdot y_{m-1} + b_m)
$$

where:
- $ y_m $ is the output of the $ m^{th} $ layer.
- $ W_m $ and $ b_m $ are the **weights** and **biases** of the $ m^{th} $ layer.
- $ f $ is an **activation function**
- The input to the first layer is the **node embedding**:



In [6]:
class RegressionModule(torch.nn.Module):
    def __init__(self, input_dim):
        super().__init__()
        self.fc1 = torch.nn.Linear(input_dim, 64)
        self.fc2 = torch.nn.Linear(64, 32)
        self.fc3 = torch.nn.Linear(32, 1)
        
    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        return self.fc3(x)

## Full Model

In [7]:
class ILGR(torch.nn.Module):
    def __init__(self, hidden_channels):
        super().__init__()
        self.embedding = ILGRNodeEmbedding(hidden_channels)
        self.regression = RegressionModule(hidden_channels * 4)  # Concatenated embeddings
        
    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        embedding = self.embedding(x, edge_index)
        return self.regression(embedding)

### Criticality Score Calculation

### Algorithm 3: Conventional Approach for Identifying Critical Nodes/Links
**Input:** Graph $ G $ with $ V $ nodes.
**Output:** Node critical scores.

**1. For each node/link** $ n $ in $ V $:
   - Remove node $ n $ from the graph $ G $.
   - Compute robustness metric of the **residual graph** $ (G - n) $.
   - Assign a **criticality score** to node $ n $.

**2. End loop**.

3. Rank nodes based on computed **criticality scores**.
4. Top ranks correspond to the **most critical nodes**.
**5. Return**: Top $ N\% $ of most critical nodes.

In [8]:
def compute_criticality_scores(graph, metric):
    scores = []
    for node in tqdm(graph.nodes(), desc="Computing Criticality Scores"):
        subgraph = graph.copy()
        subgraph.remove_node(node)
        score = metric(graph) - metric(subgraph)  # Drop in robustness
        scores.append(score)
    return scores

### Ranking Loss
$$
     L_{ij} = -f(r_{ij}) \log(σ(\hat{y}_{ij})) - (1 - f(r_{ij})) \log(1 - σ(\hat{y}_{ij}))
     $$

In [9]:
"""def pairwise_ranking_loss(y_pred, y_true):
    # Compute all pairwise differences
    diff_true = y_true.unsqueeze(1) - y_true.unsqueeze(0)  # r_ij = r_i - r_j
    diff_pred = y_pred.unsqueeze(1) - y_pred.unsqueeze(0)  # y_ij = y_i - y_j

    # Apply sigmoid function to ground truth ranking differences f(r_ij)
    f_rij = torch.sigmoid(diff_true)

    # Compute sigmoid of predicted ranking differences σ(ŷ_ij)
    sigma_y_pred = torch.sigmoid(diff_pred)

    # Compute the pairwise ranking loss
    loss = -f_rij * torch.log(sigma_y_pred + 1e-10) - (1 - f_rij) * torch.log(1 - sigma_y_pred + 1e-10)

    # Mask to consider only valid pairs (i < j) to avoid redundant comparisons
    mask = torch.triu(torch.ones_like(loss), diagonal=1).bool()
    loss = loss[mask]

    # Compute mean loss over valid pairs
    return loss.mean()"""

'def pairwise_ranking_loss(y_pred, y_true):\n    # Compute all pairwise differences\n    diff_true = y_true.unsqueeze(1) - y_true.unsqueeze(0)  # r_ij = r_i - r_j\n    diff_pred = y_pred.unsqueeze(1) - y_pred.unsqueeze(0)  # y_ij = y_i - y_j\n\n    # Apply sigmoid function to ground truth ranking differences f(r_ij)\n    f_rij = torch.sigmoid(diff_true)\n\n    # Compute sigmoid of predicted ranking differences σ(ŷ_ij)\n    sigma_y_pred = torch.sigmoid(diff_pred)\n\n    # Compute the pairwise ranking loss\n    loss = -f_rij * torch.log(sigma_y_pred + 1e-10) - (1 - f_rij) * torch.log(1 - sigma_y_pred + 1e-10)\n\n    # Mask to consider only valid pairs (i < j) to avoid redundant comparisons\n    mask = torch.triu(torch.ones_like(loss), diagonal=1).bool()\n    loss = loss[mask]\n\n    # Compute mean loss over valid pairs\n    return loss.mean()'

### Optimize Pairwise Loss Computation
Replace the nested-loop pairwise loss with a vectorized implementation to handle large graphs (other version of pairwise ranking loss) :

$$
L = \frac{1}{N(N-1)/2} \sum_{i < j} \log \left( 1 + \exp \left( - \text{sign}(y_{\text{true}}^{(i)} - y_{\text{true}}^{(j)}) \cdot (y_{\text{pred}}^{(i)} - y_{\text{pred}}^{(j)}) \right) \right)
$$

### Where:
- $ y_{\text{pred}}^{(i)} $ and $ y_{\text{pred}}^{(j)} $ are the predicted values for the $i$-th and $j$-th items, respectively.
- $ y_{\text{true}}^{(i)} $ and $ y_{\text{true}}^{(j)} $ are the true labels for the $i$-th and $j$-th items, respectively.
- $ \text{sign}(x) $ is the sign function:
- $ \text{sign}(x) = +1 $ if $ x > 0 $
- $ \text{sign}(x) = -1 $ if $ x < 0 $

### Breakdown:

- $ y_{\text{true}}^{(i)} - y_{\text{true}}^{(j)} $: The difference in the true values (target ranking).
- $ y_{\text{pred}}^{(i)} - y_{\text{pred}}^{(j)} $: The difference in the predicted values (model's ranking).
- $ \text{sign}(y_{\text{true}}^{(i)} - y_{\text{true}}^{(j)}) $ ensures that:
- If the true ranking is correct (i.e., $ y_{\text{true}}^{(i)} > y_{\text{true}}^{(j)} $), we want $ y_{\text{pred}}^{(i)} $ to be greater than $ y_{\text{pred}}^{(j)} $.
- The difference in predictions should match the expected order.

This formulation helps enforce the correct ranking order between pairs, which is critical in learning-to-rank tasks.


In [10]:
def pairwise_ranking_loss(y_pred, y_true):
    y_pred = y_pred.squeeze()
    y_true = y_true.squeeze()
    
    # Compute all pairwise differences
    diff_pred = y_pred.unsqueeze(1) - y_pred.unsqueeze(0)  # Shape [N, N]
    diff_true = y_true.unsqueeze(1) - y_true.unsqueeze(0)  # Shape [N, N]
    
    # Mask for valid pairs (i < j)
    mask = torch.triu(torch.ones_like(diff_true), diagonal=1).bool()
    diff_pred = diff_pred[mask]
    diff_true = diff_true[mask]
    
    # Compute loss
    loss = torch.log(1 + torch.exp(-torch.sign(diff_true) * diff_pred)).mean()
    return loss

## Graph Preprocessing

### Generate Synthetic Graphs

In [11]:
# Power-law graph (Barabási-Albert model)
def generate_power_law(n, m):
    return nx.barabasi_albert_graph(n, m)

# Power-law cluster graph (Holme-Kim model)
def generate_power_law_cluster(n, m, p):
    return nx.powerlaw_cluster_graph(n, m, p)

### real-world datasets (load function)

In [12]:
def load_real_world_graph(dataset_name):
    G = nx.read_edgelist(dataset_name, nodetype=int)
    return G

### Convert NetworkX graph to PyTorch Geometric format

In [13]:
def nx_to_pyg(nx_graph, criticality_scores):
    # Convert NetworkX graph to PyG format
    pyg_data = from_networkx(nx_graph)

    # Use criticality scores as node features
    pyg_data.x = criticality_scores.view(-1, 1)

    return pyg_data

## Algorithm 2: ILGR Training
**Input:** Model with unknown weights.
**Output:** Trained model.

1. Compute ground truth **criticality scores** of nodes based on graph robustness score.
2. **For each epoch do**:
   - Get each **node embedding** from the embedding module.
   - Estimate **criticality scores** of nodes/links using the regression module.
   - Update weights of both modules by solving the loss function:
     $$
     L_{ij} = -f(r_{ij}) \log(σ(\hat{y}_{ij})) - (1 - f(r_{ij})) \log(1 - σ(\hat{y}_{ij}))
     $$
3. **End loop**.
4. Predict nodescores on the test graph.
5. **Return**: Top $ N\% $ of most critical nodes.


In [14]:
def train_model(model, data, y_true, epochs=100, lr=0.001):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    data, y_true = data.to(device), y_true.to(device)
    
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)

    progress_bar = tqdm(range(epochs), desc="Training Model", dynamic_ncols=True)
    
    for epoch in progress_bar:
        model.train()
        optimizer.zero_grad()

        # Forward pass
        y_pred = model(data)

        # Compute loss
        loss = pairwise_ranking_loss(y_pred, y_true)

        # Backward pass and optimization
        loss.backward()
        optimizer.step()
        
        # Update progress bar with loss value
        progress_bar.set_postfix(loss=loss.item())


### **Evaluation Metrics: Top-N% Accuracy**

To measure the accuracy of our framework, we use **Top-N% Accuracy**, which is defined as the percentage of overlap between the predicted Top-N% nodes/links and the ground-truth Top-N% nodes/links (computed using a conventional baseline approach). 

The formula for **Top-N% Accuracy** is given by:

$$
\text{Top-N% Accuracy} = \frac{\left| \{\text{Predicted Top-N% nodes/links}\} \cap \{\text{True Top-N% nodes/links}\} \right|}{|V| \times (N/100)}
$$

where:
- $ |V| $ is the total number of nodes/links in the graph.
- $ N $ is the percentage band (e.g., Top-5%).
- $ \cap $ denotes the intersection between the predicted and true Top-N% sets.


In [15]:
def top_n_accuracy(y_pred, y_true, N=5):
    num_nodes = len(y_true)
    top_n = int(num_nodes * (N / 100))

    # Get indices of top N% nodes for predicted and true values
    top_pred = torch.argsort(y_pred.squeeze(), descending=True)[:top_n]
    top_true = torch.argsort(y_true.squeeze(), descending=True)[:top_n]

    # Compute accuracy as percentage of overlap
    accuracy = len(set(top_pred.tolist()) & set(top_true.tolist())) / top_n
    return accuracy

### **Test the framwork**

### 1000 nodes (power-law graph)

In [16]:
# Step 1: Generate a power-law graph
G = generate_power_law(n=1000, m=3)

# Step 2: Compute criticality scores
y_true = compute_criticality_scores(G, compute_weighted_spectrum)
y_true = torch.tensor(y_true, dtype=torch.float)

# Step 3: Convert to PyG format with node features
data = nx_to_pyg(G, y_true)

# Step 4: Define the model
hidden_dim = 32
model = ILGR(hidden_dim)
print(model)

# Step 5: Train the model
train_model(model, data, y_true, epochs=1000, lr=0.001)

# Step 6: Evaluate the model
model.eval()
y_pred = model(data).detach()  # Ensure no gradients

# Save the trained model
torch.save(model.state_dict(), "trained_model_pl.pth")
print("Model saved successfully!")

# Evaluate the top-N% nodes based on their criticality scores (true values)
top_n = int(len(y_true) * 0.05)  # Top 5% nodes
top_n_true_indices = torch.argsort(y_true.squeeze(), descending=True)[:top_n]
top_n_pred_indices = torch.argsort(y_pred.squeeze(), descending=True)[:top_n]

print("Top-5% True Node Indices:", top_n_true_indices.tolist())
print("Top-5% Predicted Node Indices:", top_n_pred_indices.tolist())

# Display top-5% criticality scores for both true and predicted values
# print("Top-5% True Criticality Scores:", y_true[top_n_true_indices].tolist())
# print("Top-5% Predicted Criticality Scores:", y_pred[top_n_pred_indices].tolist())


accuracy = top_n_accuracy(y_pred, y_true, N=5)  # Top-5% accuracy
print(f"Top-5% Accuracy: {accuracy * 100:.2f}%")

Computing Criticality Scores: 100%|██████████| 1000/1000 [03:34<00:00,  4.67it/s]


ILGR(
  (embedding): ILGRNodeEmbedding(
    (conv1): SAGEConv(1, 32, aggr=mean)
    (conv2): SAGEConv(32, 32, aggr=mean)
    (conv3): SAGEConv(32, 32, aggr=mean)
    (attention): MultiheadAttention(
      (out_proj): NonDynamicallyQuantizableLinear(in_features=32, out_features=32, bias=True)
    )
  )
  (regression): RegressionModule(
    (fc1): Linear(in_features=128, out_features=64, bias=True)
    (fc2): Linear(in_features=64, out_features=32, bias=True)
    (fc3): Linear(in_features=32, out_features=1, bias=True)
  )
)


Training Model: 100%|██████████| 1000/1000 [00:08<00:00, 124.36it/s, loss=0.00422]


Model saved successfully!
Top-5% True Node Indices: [68, 890, 573, 731, 55, 544, 19, 102, 712, 146, 28, 301, 446, 720, 381, 729, 11, 4, 25, 17, 411, 10, 369, 64, 89, 152, 233, 261, 409, 8, 258, 279, 217, 90, 186, 128, 92, 3, 12, 2, 32, 191, 917, 560, 39, 242, 127, 559, 870, 331]
Top-5% Predicted Node Indices: [68, 890, 573, 731, 55, 544, 19, 102, 712, 146, 28, 301, 446, 720, 381, 729, 11, 4, 25, 17, 411, 10, 369, 64, 89, 152, 233, 261, 409, 8, 258, 279, 217, 90, 186, 128, 92, 3, 12, 2, 32, 191, 917, 560, 39, 242, 127, 559, 870, 331]
Top-5% Accuracy: 100.00%


### 1000 nodes (power-law cluster graph)

In [17]:
# Step 1: Generate a power-law graph cluster
G = generate_power_law_cluster(n=1000, m=3, p=0.3)

# Step 2: Compute criticality scores
y_true = compute_criticality_scores(G, compute_weighted_spectrum)
y_true = torch.tensor(y_true, dtype=torch.float)

# Step 3: Convert to PyG format with node features
data = nx_to_pyg(G, y_true)

# Step 4: Define the model
hidden_dim = 32
model = ILGR(hidden_dim)
print(model)

# Step 5: Train the model
train_model(model, data, y_true, epochs=1000, lr=0.001)

# Step 6: Evaluate the model
model.eval()
y_pred = model(data).detach()  # Ensure no gradients

# Save the trained model
torch.save(model.state_dict(), "trained_model_plc.pth")
print("Model saved successfully!")


# Evaluate the top-N% nodes based on their criticality scores (true values)
top_n = int(len(y_true) * 0.05)  # Top 5% nodes
top_n_true_indices = torch.argsort(y_true.squeeze(), descending=True)[:top_n]
top_n_pred_indices = torch.argsort(y_pred.squeeze(), descending=True)[:top_n]

print("Top-5% True Node Indices:", top_n_true_indices.tolist())
print("Top-5% Predicted Node Indices:", top_n_pred_indices.tolist())

# Display top-5% criticality scores for both true and predicted values
# print("Top-5% True Criticality Scores:", y_true[top_n_true_indices].tolist())
# print("Top-5% Predicted Criticality Scores:", y_pred[top_n_pred_indices].tolist())


accuracy = top_n_accuracy(y_pred, y_true, N=5)  # Top-5% accuracy
print(f"Top-5% Accuracy: {accuracy * 100:.2f}%")

Computing Criticality Scores: 100%|██████████| 1000/1000 [03:33<00:00,  4.69it/s]


ILGR(
  (embedding): ILGRNodeEmbedding(
    (conv1): SAGEConv(1, 32, aggr=mean)
    (conv2): SAGEConv(32, 32, aggr=mean)
    (conv3): SAGEConv(32, 32, aggr=mean)
    (attention): MultiheadAttention(
      (out_proj): NonDynamicallyQuantizableLinear(in_features=32, out_features=32, bias=True)
    )
  )
  (regression): RegressionModule(
    (fc1): Linear(in_features=128, out_features=64, bias=True)
    (fc2): Linear(in_features=64, out_features=32, bias=True)
    (fc3): Linear(in_features=32, out_features=1, bias=True)
  )
)


Training Model: 100%|██████████| 1000/1000 [00:06<00:00, 143.02it/s, loss=0.00224]


Model saved successfully!
Top-5% True Node Indices: [557, 570, 707, 749, 432, 466, 684, 589, 100, 672, 404, 674, 829, 492, 446, 632, 983, 818, 914, 464, 590, 228, 799, 912, 105, 289, 239, 595, 904, 886, 162, 591, 792, 476, 298, 299, 393, 809, 236, 918, 567, 816, 39, 249, 917, 47, 676, 207, 284, 338]
Top-5% Predicted Node Indices: [557, 570, 707, 749, 432, 466, 684, 589, 100, 672, 404, 674, 829, 492, 446, 632, 983, 818, 914, 464, 590, 228, 799, 912, 105, 289, 239, 595, 904, 162, 886, 591, 476, 792, 298, 299, 393, 809, 236, 918, 567, 816, 39, 249, 917, 47, 676, 207, 284, 338]
Top-5% Accuracy: 100.00%


### Bio Yeast Dataset (power_law_ws)

In [18]:
# Load the Matrix Market file (Bio Yeast graph)
file_path = "/kaggle/input/bio-yeast/bio-yeast.mtx"
matrix = mmread(file_path)

# Convert to a NetworkX graph
# Use the appropriate function based on your NetworkX version
try:
    G = nx.from_scipy_sparse_array(matrix)  # For newer versions of NetworkX
except AttributeError:
    G = nx.from_scipy_sparse_matrix(matrix)  # For older versions of NetworkX

# Compute criticality scores (assuming your function is defined)
y_true = compute_criticality_scores(G, compute_weighted_spectrum)
y_true = torch.tensor(y_true, dtype=torch.float)

# Convert to PyG format
data = nx_to_pyg(G, y_true)

print(f"Loaded Bio Yeast graph with {G.number_of_nodes()} nodes and {G.number_of_edges()} edges.")

# Define model architecture (must match saved model)
hidden_dim = 32
model = ILGR(hidden_dim)

# Load trained weights
model.load_state_dict(torch.load("/kaggle/working/trained_model_pl.pth", weights_only=True))
model.eval()  # Set to evaluation model
print("Model loaded successfully!")

# Make predictions
y_pred = model(data).detach()

# Evaluate Top-5% Nodes
top_n = int(len(y_true) * 0.05)
top_n_true_indices = torch.argsort(y_true.squeeze(), descending=True)[:top_n]
top_n_pred_indices = torch.argsort(y_pred.squeeze(), descending=True)[:top_n]

print("Top-5% True Node Indices:", top_n_true_indices.tolist()) 
print("Top-5% Predicted Node Indices:", top_n_pred_indices.tolist())

# Compute Top-5% Accuracy
accuracy = top_n_accuracy(y_pred, y_true, N=5)
print(f"Top-5% Accuracy: {accuracy * 100:.2f}%")

Computing Criticality Scores: 100%|██████████| 1458/1458 [12:06<00:00,  2.01it/s]


Loaded Bio Yeast graph with 1458 nodes and 1948 edges.
Model loaded successfully!
Top-5% True Node Indices: [542, 1078, 949, 640, 787, 81, 285, 714, 88, 725, 797, 829, 91, 964, 943, 1032, 645, 888, 1, 595, 155, 278, 368, 982, 996, 458, 417, 196, 268, 1027, 1085, 428, 702, 473, 908, 963, 766, 669, 567, 6, 1140, 984, 393, 869, 242, 1043, 937, 560, 406, 398, 63, 2, 850, 30, 1025, 56, 109, 1400, 484, 250, 1114, 890, 1126, 1409, 379, 31, 929, 1115, 517, 251, 201, 1440]
Top-5% Predicted Node Indices: [936, 1437, 1081, 989, 348, 439, 481, 1147, 804, 781, 1348, 1347, 25, 291, 1151, 386, 1421, 709, 76, 640, 1078, 949, 542, 88, 81, 787, 194, 1030, 245, 676, 1048, 1021, 623, 917, 176, 796, 1006, 998, 991, 134, 980, 977, 740, 116, 100, 762, 896, 299, 556, 909, 546, 543, 1096, 530, 839, 311, 1108, 851, 479, 477, 446, 1152, 1168, 298, 41, 1057, 906, 39, 588, 33, 294, 296]
Top-5% Accuracy: 9.72%


### Bio Yeast Dataset (power_law_cluster_ws)

In [19]:
# Load the Matrix Market file (Bio Yeast graph)
file_path = "/kaggle/input/bio-yeast/bio-yeast.mtx"
matrix = mmread(file_path)

# Convert to a NetworkX graph
# Use the appropriate function based on your NetworkX version
try:
    G = nx.from_scipy_sparse_array(matrix)  # For newer versions of NetworkX
except AttributeError:
    G = nx.from_scipy_sparse_matrix(matrix)  # For older versions of NetworkX

# Compute criticality scores (assuming your function is defined)
y_true = compute_criticality_scores(G, compute_weighted_spectrum)
y_true = torch.tensor(y_true, dtype=torch.float)

# Convert to PyG format
data = nx_to_pyg(G, y_true)

print(f"Loaded Bio Yeast graph with {G.number_of_nodes()} nodes and {G.number_of_edges()} edges.")

# Define model architecture (must match saved model)
hidden_dim = 32
model = ILGR(hidden_dim)

# Load trained weights
model.load_state_dict(torch.load("/kaggle/working/trained_model_plc.pth", weights_only=True))
model.eval()  # Set to evaluation model
print("Model loaded successfully!")

# Make predictions
y_pred = model(data).detach()

# Evaluate Top-5% Nodes
top_n = int(len(y_true) * 0.05)
top_n_true_indices = torch.argsort(y_true.squeeze(), descending=True)[:top_n]
top_n_pred_indices = torch.argsort(y_pred.squeeze(), descending=True)[:top_n]

print("Top-5% True Node Indices:", top_n_true_indices.tolist()) 
print("Top-5% Predicted Node Indices:", top_n_pred_indices.tolist())

# Compute Top-5% Accuracy
accuracy = top_n_accuracy(y_pred, y_true, N=5)
print(f"Top-5% Accuracy: {accuracy * 100:.2f}%")

Computing Criticality Scores: 100%|██████████| 1458/1458 [10:37<00:00,  2.29it/s]

Loaded Bio Yeast graph with 1458 nodes and 1948 edges.
Model loaded successfully!
Top-5% True Node Indices: [542, 1078, 949, 640, 787, 81, 285, 714, 88, 725, 797, 829, 91, 964, 943, 1032, 645, 888, 1, 595, 155, 278, 368, 982, 996, 458, 417, 196, 268, 1027, 1085, 428, 702, 473, 908, 963, 766, 669, 567, 6, 1140, 984, 393, 869, 242, 1043, 937, 560, 406, 398, 63, 2, 850, 30, 1025, 56, 109, 1400, 484, 250, 1114, 890, 1126, 1409, 379, 31, 929, 1115, 517, 251, 201, 1440]
Top-5% Predicted Node Indices: [640, 1078, 285, 714, 542, 949, 81, 787, 88, 91, 829, 595, 645, 1032, 1, 155, 1027, 908, 888, 368, 196, 428, 458, 797, 725, 964, 1085, 996, 963, 766, 702, 6, 1140, 2, 943, 1043, 560, 937, 982, 398, 406, 268, 869, 278, 484, 379, 31, 393, 567, 1440, 1400, 687, 1126, 1387, 1409, 517, 557, 958, 438, 417, 1402, 769, 984, 421, 669, 1115, 1127, 30, 50, 1232, 126, 1435]
Top-5% Accuracy: 81.94%


### US Power Grid Dataset

In [20]:
"""# Load the Matrix Market file (US Power Grid graph)
file_path = "/kaggle/input/power-us-grid/power-US-Grid.mtx"
matrix = mmread(file_path)

# Convert to a NetworkX graph
# Use the appropriate function based on your NetworkX version
try:
    G = nx.from_scipy_sparse_array(matrix)  # For newer versions of NetworkX
except AttributeError:
    G = nx.from_scipy_sparse_matrix(matrix)  # For older versions of NetworkX

# Compute criticality scores (assuming your function is defined)
y_true = compute_criticality_scores(G, compute_effective_resistance)
y_true = torch.tensor(y_true, dtype=torch.float)

# Convert to PyG format
data = nx_to_pyg(G, y_true)

print(f"Loaded US Power Grid graph with {G.number_of_nodes()} nodes and {G.number_of_edges()} edges.")

# Define model architecture (must match saved model)
hidden_dim = 32
model = ILGR(hidden_dim)

# Load trained weights
model.load_state_dict(torch.load("/kaggle/working/trained_model_pl.pth"))
model.eval()  # Set to evaluation model
print("Model loaded successfully!")

# Make predictions
y_pred = model(data).detach()

# Evaluate Top-5% Nodes
top_n = int(len(y_true) * 0.05)
top_n_true_indices = torch.argsort(y_true.squeeze(), descending=True)[:top_n]
top_n_pred_indices = torch.argsort(y_pred.squeeze(), descending=True)[:top_n]

print("Top-5% True Node Indices:", top_n_true_indices.tolist())
print("Top-5% Predicted Node Indices:", top_n_pred_indices.tolist())

# Compute Top-5% Accuracy
accuracy = top_n_accuracy(y_pred, y_true, N=5)
print(f"Top-5% Accuracy: {accuracy * 100:.2f}%")"""

'# Load the Matrix Market file (US Power Grid graph)\nfile_path = "/kaggle/input/power-us-grid/power-US-Grid.mtx"\nmatrix = mmread(file_path)\n\n# Convert to a NetworkX graph\n# Use the appropriate function based on your NetworkX version\ntry:\n    G = nx.from_scipy_sparse_array(matrix)  # For newer versions of NetworkX\nexcept AttributeError:\n    G = nx.from_scipy_sparse_matrix(matrix)  # For older versions of NetworkX\n\n# Compute criticality scores (assuming your function is defined)\ny_true = compute_criticality_scores(G, compute_effective_resistance)\ny_true = torch.tensor(y_true, dtype=torch.float)\n\n# Convert to PyG format\ndata = nx_to_pyg(G, y_true)\n\nprint(f"Loaded US Power Grid graph with {G.number_of_nodes()} nodes and {G.number_of_edges()} edges.")\n\n# Define model architecture (must match saved model)\nhidden_dim = 32\nmodel = ILGR(hidden_dim)\n\n# Load trained weights\nmodel.load_state_dict(torch.load("/kaggle/working/trained_model_pl.pth"))\nmodel.eval()  # Set to e